In [ ]:
import os, glob
import pandas as pd
from haversine import haversine

path = 'data/cabspottingdata/Multiple_Trajectories/'
save = 'data/cabspottingdata/Multiple_Trajectories/Single_Trajectory'

file_list = os.listdir(path)
f_name=[]
for filename in file_list:
    if os.path.splitext(filename)[1] == '.csv':
        f_name.append(os.path.splitext(filename)[0])
        
for t in f_name:
    df3 = pd.DataFrame(columns=("CarName", "LineString", "StartTime", "Num_p"))
    
    df = pd.read_csv(path+t+'.csv', index_col=0)
    df2 = df.sort_index(ascending=False)
    df2 = df2.reset_index()
    df2 = df2.drop(columns=['index'])
    df2 = df2.loc[(df2['latitude'] <= 37.811778) & (df2['latitude'] >= 37.705258)]
    df2 = df2.loc[(df2['longitude'] <= -122.351824) & (df2['longitude'] >= -122.526568)]
    
    
    flag = df2.iloc[0]['p']
    stime = df2.iloc[0]['time']
    line = 'LINESTRING('
    start = (35.1, -124.1)
    cnt = 0
    dis = 0

    for la, lo, p, ti in zip(df2["latitude"], df2["longitude"], df2["p"], df2["time"]):        
        if flag == p:
            goal = (la, lo)
            dis = haversine(start, goal, unit = 'm')
            if dis >= 150:
                line += str(lo) + ' ' + str(la) + ','
                cnt = cnt + 1
                start = goal
        else:
            line = line[:-1]
            line += ')'
            df3 = df3.append({'CarName' : t, 'LineString' : line, 'StartTime' : stime, 'Num_p' : cnt}, ignore_index=True)
            line = 'LINESTRING('
            line += str(lo) + ' ' + str(la) + ','
            cnt = 0
            stime = ti
            flag = p
    line = line[:-1]
    line += ')'
    df3 = df3.append({'CarName' : t, 'LineString' : line, 'StartTime' : stime, 'Num_p' : cnt}, ignore_index=True)
    df3 = df3.loc[df3['Num_p'] >= 5]
    df3 = df3.loc[df3['Num_p'] <= 110]
    df3.to_csv(save + '/' + t + '_tra.csv')
    df3 = pd.DataFrame(columns=("CarName", "LineString", "StartTime", "Num_p"))